<a href="https://colab.research.google.com/github/rabinatwayana/DL_torchgeo_MMFlood_Segmentation/blob/master/Drive_mmflood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning-based Semantic Flood Mapping using UNet and SegFormer on Multimodal Earth Observation Data

## Install Packages

In [1]:
!pip install torchgeo

ModuleNotFoundError: No module named 'torchgeo'

## Import Packages

In [ ]:
import torchgeo

## Dataset
MMFlood dataset is a multimodal flood delineation dataset (Montello et al., 2022).

 Some Sentinel-1 tiles have missing data, which are automatically set to 0. Corresponding pixels in masks are set to 255 and should be ignored in performance computation.

Dataset features:

- 1,748 Sentinel-1 tiles of varying pixel dimensions

- Multimodal dataset (Sentinel-1, DEMs and hydrography maps (available for 1,012 tiles out of 1,748))

- 95 flood events from 42 different countries

- Flood delineation maps (ground truth) is obtained from Copernicus EMS

- Missing data in Sentinel-1 tiles are set to 0 and corrsponding pixels in masks are set to 255 (must ignored in performance computation)  

Dataset classes:

- no flood

- flood



## References
Montello, F., Arnaudo, E., & Rossi, C. (2022). MMFlood: A Multimodal Dataset for Flood Delineation From Satellite Imagery. IEEE Access, 10, 96774–96787. https://doi.org/10.1109/ACCESS.2022.3205419

